In [1]:
import numpy as np
import scipy
import pandas as pd
import matplotlib.pyplot as plt

import os 

import string 
from scipy.stats import norm
import torch 
#import torchsort 
import copy 
import torch.nn.functional as F

import torch.distributions as dist
from sklearn.linear_model import LinearRegression

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

from dataset_prepare import * 
torch.autograd.set_detect_anomaly(True)

from probaforms.models import CVAE

import argparse
parser = argparse.ArgumentParser()
parser.add_argument('--n', type=int, default=2000)
parser.add_argument('--d', type=int, default=1)
parser.add_argument('--nval', type=int, default=1000)
parser.add_argument('--alpha', type=float, default=0.1)
parser.add_argument('--niter', type=int, default=100)
parser.add_argument('--densitymodel', type=str, default='CVAE')
parser.add_argument('--dataset', type=str, default='meps_19')
parser.add_argument('--lamb', type=float, default=100)
parser.add_argument('--model', type=str, default='linear')
parser.add_argument('--conformalscore', type=str, default='residual')
parser.add_argument('--wsc_delta', type=float, default=0.1)

args = parser.parse_args([])
n = args.n
d = args.d
alpha = args.alpha
niter = args.niter
model = args.model
densitymodel = args.densitymodel
dataset_name = args.dataset


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#device = 'cpu'
print('device is ', device)

def conformalScore(Y, Yhat, sd = 1):
    if args.conformalscore == 'residual':
        score = np.abs(Yhat - Y)
    elif args.conformalscore == 'normalized':
        score = np.abs(Yhat - Y) / sd
    return score

def conformalScore_torch(Y, Yhat, sd = 1):
    if args.conformalscore == 'residual':
        score = torch.abs(Yhat - Y)
    elif args.conformalscore == 'normalized':
        score = torch.abs(Yhat - Y) / sd
    return score


outfun = LinearRegression()

# torch linear model 
class LinearModel(torch.nn.Module):
    def __init__(self, d):
        super(LinearModel, self).__init__()
        self.linear = torch.nn.Linear(d, 1)
        torch.nn.init.xavier_uniform_(self.linear.weight)
    def forward(self, x):
        y_pred = self.linear(x)
        return y_pred
    

# a MLP with LeakyReLU activation
class MLP(torch.nn.Module):
    def __init__(self, d):
        super(MLP, self).__init__()
        hidd = 16
        self.linear1 = torch.nn.Linear(d, hidd)
        self.linear2 = torch.nn.Linear(hidd, hidd)
        self.linear3 = torch.nn.Linear(hidd, 1)
        self.leakyrelu = torch.nn.LeakyReLU()
        self.relu = torch.nn.ReLU()
        # initialize the weights
        torch.nn.init.xavier_uniform_(self.linear1.weight)
        torch.nn.init.xavier_uniform_(self.linear2.weight)
        torch.nn.init.xavier_uniform_(self.linear3.weight)
        
    def forward(self, x):
        y_pred = self.linear1(x)
        y_pred = self.leakyrelu(y_pred)
        y_pred = self.linear2(y_pred)
        y_pred = self.leakyrelu(y_pred)
        y_pred = self.linear3(y_pred)
        return y_pred
    
    
# a class for the torch linear model with fit and predict methods
class TorchBaseModel():
    def __init__(self, d):
        if args.model == 'linear' or args.model == 'linear_cc':
            self.model = LinearModel(d).to(device)
        elif args.model == 'mlp' or args.model == 'mlp_cc':
            self.model = MLP(d).to(device)
        self.criterion = torch.nn.MSELoss(reduction='mean')
        #self.optimizer = torch.optim.SGD(self.model.parameters(), lr=1e-3)
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=1e-3, weight_decay=1e-4)
        
    def fit(self, X, Y):
        X = torch.from_numpy(X).float().to(device)
        Y = torch.from_numpy(Y).float().to(device)
        for t in range(10000):
            # Forward pass: Compute predicted y by passing x to the model
            y_pred = self.model(X)
            # Compute and print loss
            loss = ((y_pred.reshape(-1) - Y)**2).mean()
            # Zero gradients, perform a backward pass, and update the weights.
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
            if t % 100 == 99:
                print(t, loss.item())
        return self
    
    def predict(self, X):
        X = torch.from_numpy(X).float().to(device)
        return self.model(X).detach().cpu().numpy().reshape(-1)

class TorchLinearModel_CC():
    def __init__(self, d, lamb = 1, density_model = None):
        if args.model == 'linear_cc':
            self.model = LinearModel(d).to(device)
        elif args.model == 'mlp_cc':
            self.model = MLP(d).to(device)
        self.lamb = lamb
        self.criterion = torch.nn.MSELoss(reduction='mean')
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=1e-3)
        self.density_model = density_model
        
    def fit(self, X, Y, X_CC, Y_CC, Xtest, Ytest, sen, i = 0):
        # plot the data
        X = torch.from_numpy(X).float().to(device)
        Y = torch.from_numpy(Y).float().to(device)
        Xval = X_CC
        Yval = Y_CC
        X_CC = torch.from_numpy(X_CC).float().to(device)
        Y_CC = torch.from_numpy(Y_CC).float().to(device)
        X_CC = X
        Y_CC = Y
        
        #first train a simple linear model
        for t in range(10000):
            # Forward pass: Compute predicted y by passing x to the model
            y_pred = self.model(X)
            # Compute and print loss
            loss = ((y_pred.reshape(-1) - Y)**2).mean()
            
            # Zero gradients, perform a backward pass, and update the weights.
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
        
        nsamples = 2000
        samples = np.zeros((X_CC.shape[0], nsamples))
        for i in range(nsamples):
            condX = X_CC.cpu().numpy()
            samples[:,i] = self.density_model.sample(condX).reshape(-1)
        sd_cc = np.std(samples, axis = 1)
        sd_cc = torch.from_numpy(sd_cc).float().to(device)

        samples_test = np.zeros((Xtest.shape[0], nsamples))
        for i in range(nsamples):
            condX = Xtest
            samples_test[:,i] = self.density_model.sample(condX).reshape(-1)
        sd_test = np.std(samples_test, axis = 1)
        sd_test = torch.from_numpy(sd_test).float().to(device)

        best_loss = 1e7
        improved_in_last = 0
        for t in range(60):
            # train a density model 
            y_pred = self.model(X_CC)
            V_CC = conformalScore_torch(Y_CC, y_pred.reshape(-1), sd = sd_cc)
            
            y_pred = self.model(X)
            loss = ((y_pred.reshape(-1) - Y)**2).mean()
            
            y_pred = self.model(X_CC)
            V_CC = conformalScore_torch(Y_CC, y_pred.reshape(-1), sd = sd_cc)
            
            wdiv = 0 
            nsamples = 500
            nsamples = min(nsamples, X_CC.shape[0])
            batch_size = 100
            temperature = 10
            # random sample nsamples from X_CC
            rand_index = np.random.choice(X_CC.shape[0], nsamples)
            wdiv = torch.zeros_like(V_CC[:nsamples])
            for indi, i in enumerate(rand_index):
                if densitymodel == 'mdn':
                    thissampled = torch.Tensor(self.density_model.sample((torch.ones_like(X_CC) * X_CC[i,])[:nsamples].cpu().numpy())[1].reshape(-1))
                else:
                    condX = (torch.ones_like(X_CC) * X_CC[i,])[:nsamples].cpu().numpy()
                    #thissampled = self.density_model.generate(condX.shape[0], cond = condX).values.reshape(-1)
                    thissampled = self.density_model.sample(condX).reshape(-1)
                    thissampled = torch.Tensor(thissampled)
                thissampled = thissampled.to(device)
                sample_index = np.random.choice(X_CC.shape[0], nsamples)
                VCC_sampled = V_CC[sample_index]
                V_givenx = conformalScore_torch(thissampled, torch.ones_like(y_pred.reshape(-1)[:nsamples]) * y_pred[i,], \
                                sd = torch.ones_like(y_pred.reshape(-1)[:nsamples]) * sd_cc[i])

                diff = approx_ecdf(V_givenx, VCC_sampled, grid_size = 100, temperature = 10)
                # when the max cannot be reduced effectively, minimize the softmax is still effective for improving the cc
                wdiv[indi] = (diff * F.softmax(diff * temperature, dim = 0)).sum()

            div = wdiv * F.softmax(wdiv * temperature, dim = 0)
            div = div.sum()
            loss1 = loss + self.lamb * div.mean()
            print('t = ', t, 'loss = ', loss, 'div = ', div, 'loss1 = ', loss1)

            # Zero gradients, perform a backward pass, and update the weights.
            self.optimizer.zero_grad()
            loss1.backward()
            self.optimizer.step()

            if loss1 < best_loss:
                best_loss = loss1
                best_model = copy.deepcopy(self.model)
                improved_in_last = 0
            
            if (t+1) % 20 == 0:
                alpha = 0.1 
                Yhat = self.predict(Xval)
                Yscore = conformalScore(Yval, Yhat, sd = sd_cc.cpu().numpy())

                Yhat_test = self.predict(Xtest)
                
                nval = Xval.shape[0]
                qhat = np.quantile(Yscore, np.ceil((nval+1)*(1-alpha))/nval, interpolation='higher')
                Yslack = qhat

                print(f'cutoff is {Yslack}')
                if args.conformalscore == 'residual':
                    Ylo = Yhat_test - Yslack
                    Yup = Yhat_test + Yslack
                elif args.conformalscore == 'normalized':
                    Ylo = Yhat_test - Yslack * sd_test.cpu().numpy()
                    Yup = Yhat_test + Yslack * sd_test.cpu().numpy()

                CI = pd.DataFrame({"lower": Ylo.reshape(-1), "upper": Yup.reshape(-1)})

                cover = (Ylo <= Ytest) & (Ytest <= Yup)
                cc = 100
                for eachs in set(sen):
                    cc = min(cc, cover[sen == eachs].mean())

                pred = np.concatenate([Ylo.reshape(-1,1), Yup.reshape(-1,1)], axis = 1)
                wslab = wsc_unbiased(Xtest, Ytest, pred, delta=args.wsc_delta, M=max(200,Xtest.shape[1]+50), test_size=0.5, random_state=2020, verbose=False)
                print(t, loss.item())
                print(f'mc is {np.mean((Ylo <= Ytest) & (Ytest <= Yup))}')
                print(f'cc is {cc}')
                print(f'wsc is {wslab}')
        
        y_pred = self.model(X_CC).reshape(-1)
        V_CC = conformalScore_torch(Y_CC, y_pred)
        # plot V_CC
        self.model = best_model

        return self
    
    def predict(self, X):
        X = torch.from_numpy(X).float().to(device)
        return self.model(X).detach().cpu().numpy().reshape(-1)

/home/ruijiang/.conda/envs/conformalcc/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


device is  cuda:0


In [2]:
import numpy as np
from sklearn.model_selection import train_test_split
from tqdm import tqdm

import random 
def seed_everything(seed=0):
    random.seed(seed)
    np.random.seed(seed)


def wsc(X, y, pred, delta=0.1, M=1000, verbose=False):
    # Extract lower and upper prediction bands
    pred_l = np.min(pred,1)
    pred_h = np.max(pred,1)

    def wsc_v(X, y, pred_l, pred_h, delta, v):
        n = len(y)
        cover = (y>=pred_l)*(y<=pred_h)
        z = np.dot(X,v)
        # Compute mass
        z_order = np.argsort(z)
        z_sorted = z[z_order]
        cover_ordered = cover[z_order]
        ai_max = int(np.round((1.0-delta)*n))
        ai_best = 0
        bi_best = n-1
        cover_min = np.mean((y >= pred_l)*(y <= pred_h))
        for ai in np.arange(0, ai_max):
            bi_min = np.minimum(ai+int(np.round(delta*n)),n)
            coverage = np.cumsum(cover_ordered[ai:n]) / np.arange(1,n-ai+1)
            coverage[np.arange(0,bi_min-ai)]=1
            bi_star = ai+np.argmin(coverage)
            cover_star = coverage[bi_star-ai]
            if cover_star < cover_min:
                ai_best = ai
                bi_best = bi_star
                cover_min = cover_star
        return cover_min, z_sorted[ai_best], z_sorted[bi_best]

    def sample_sphere(n, p):
        v = np.random.randn(p, n)
        v /= np.linalg.norm(v, axis=0)
        v = v.T
        return v

    seed_everything(2020)
    V = sample_sphere(M, p=X.shape[1])
    wsc_list = [[]] * M
    a_list = [[]] * M
    b_list = [[]] * M
    if verbose:
        for m in tqdm(range(M)):
            wsc_list[m], a_list[m], b_list[m] = wsc_v(X, y, pred_l, pred_h, delta, V[m])
    else:
        for m in range(M):
            wsc_list[m], a_list[m], b_list[m] = wsc_v(X, y, pred_l, pred_h, delta, V[m])
        
    idx_star = np.argmin(np.array(wsc_list))
    a_star = a_list[idx_star]
    b_star = b_list[idx_star]
    v_star = V[idx_star]
    wsc_star = wsc_list[idx_star]
    return wsc_star, v_star, a_star, b_star

def wsc_unbiased(X, y, pred, delta=0.1, M=1000, test_size=0.75, random_state=2020, verbose=False):
    test_size = 0.75
    M = 1000
    def wsc_vab(X, y, pred, v, a, b):
        # Extract lower and upper prediction bands
        pred_l = np.min(pred,1)
        pred_h = np.max(pred,1)
        n = len(y)
        cover = (y>=pred_l)*(y<=pred_h)
        z = np.dot(X,v)
        idx = np.where((z>=a)*(z<=b))
        coverage = np.mean(cover[idx])
        return coverage

    seed_everything(random_state)
    X_train, X_test, y_train, y_test, pred_train, pred_test = train_test_split(X, y, pred, test_size=test_size,
                                                                         random_state=random_state)
    # Find adversarial parameters
    wsc_star, v_star, a_star, b_star = wsc(X_train, y_train, pred_train, delta=delta, M=M, verbose=verbose)
    # Estimate coverage
    coverage = wsc_vab(X_test, y_test, pred_test, v_star, a_star, b_star)
    return coverage


import random 
def seed_everything(seed=0):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

def approx_ecdf(points1, points2, grid_size = 100, temperature = 5):
    tgrid = torch.linspace(min(min(points1), min(points2)).item()-0.1, max(max(points1),max(points2)).item()+0.1, grid_size)
    tgrid = tgrid.to(device)
    cdf1 = (1-torch.sigmoid(temperature * (points1.reshape(-1,1) - tgrid))).mean(0)
    cdf2 = (1-torch.sigmoid(temperature * (points2.reshape(-1,1) - tgrid))).mean(0)
    diff = torch.abs(cdf1 - cdf2)
    return diff 

coverage = []
conditional_coverage = []
set_size = []
alpha_grid = np.linspace(0.1, 0.9, 10)
results = pd.DataFrame({"iter": [], "alpha": [], "coverage": [], "set_size": [], "conditional_coverage": [], 'mse': [], 'wslab': []})
mdn_results = pd.DataFrame({"iter": [], "alpha": [], "coverage": [], "set_size": [], "conditional_coverage": [], 'mse': [], 'wslab': []})
base_results = pd.DataFrame({"iter": [], "alpha": [], "coverage": [], "set_size": [], "conditional_coverage": [], 'mse': [], 'wslab': []})

In [3]:
# mlp model 
seed_everything(0)
X, Y = GetDataset(dataset_name, './data/')
Y += 1e-3 * np.random.normal(size=Y.shape)

basemodel = TorchBaseModel(X.shape[1])

index = np.random.permutation(X.shape[0])
print(f'dataset size is {X.shape[0]}, number of features is {X.shape[1]}.')
nval = int(X.shape[0] * 0.2)
ntrain = int(X.shape[0] * 0.6)
Xval = X[index[:nval],:]
Yval = Y[index[:nval]]
Xtest = X[index[(nval+ntrain):],:]
Ytest = Y[index[(nval+ntrain):]]
X = X[index[(nval):(nval+ntrain)],:]
Y = Y[index[(nval):(nval+ntrain)]]
print(f'train size is {X.shape[0]}, val size is {Xval.shape[0]}, test size is {Xtest.shape[0]}.')
scaler = StandardScaler()
scaler.fit(X)

y_scaler = StandardScaler()
y_scaler.fit(Y.reshape(-1,1))

Y = y_scaler.transform(Y.reshape(-1,1)).reshape(-1)
Yval = y_scaler.transform(Yval.reshape(-1,1)).reshape(-1)
Ytest = y_scaler.transform(Ytest.reshape(-1,1)).reshape(-1)

if dataset_name == 'meps_19' or dataset_name == 'meps_20' or dataset_name == 'meps_21':
    # define a subgroup - checking some surrogate variable
    sen = np.zeros_like(Ytest)
    sen[(Xtest[:, 9] == 1) & (Xtest[:, -1] == 1)] = 0
    sen[(Xtest[:, 9] == 1) & (Xtest[:, -1] == 0)] = 1
    sen[(Xtest[:, 9] == 0) & (Xtest[:, -1] == 1)] = 2
    sen[(Xtest[:, 9] == 0) & (Xtest[:, -1] == 0)] = 3
    
X = scaler.transform(X)
Xval = scaler.transform(Xval)
Xtest = scaler.transform(Xtest)

if (model == 'linear' or model == 'mlp') and args.conformalscore == 'residual':
    pass 
elif args.densitymodel == 'cvae':
    density_model = CVAE(n_epochs=500, latent_dim=8, hidden = (32,32), batch_size=128, lr=1e-3, weight_decay=1e-4, verbose = 1)
    density_model.fit(Y.reshape(-1,1), X)

print('density model fitting done.')
nval = Xval.shape[0]

nsamples = 1000

if (model == 'linear' or model == 'mlp') and args.conformalscore == 'residual':
    val_std = 1
    test_std = 1
else:
    samples_val = np.zeros((nval, nsamples))
    for i in range(nsamples):
        #samples_val[:,i] = density_model.sample(Xval)[1].reshape(-1)
        #samples_val[:,i] = density_model.generate(Xval.shape[0], cond = Xval).values.reshape(-1)
        samples_val[:,i] = density_model.sample(Xval).reshape(-1)

    val_std = np.std(samples_val, axis = 1)

    ntest = Xtest.shape[0]
    samples = np.zeros((ntest, nsamples))
    for i in range(nsamples):
        #samples[:,i] = density_model.sample(Xtest)[1].reshape(-1)
        #samples[:,i] = density_model.generate(Xtest.shape[0], cond = Xtest).values.reshape(-1)
        samples[:,i] = density_model.sample(Xtest).reshape(-1)

    test_std = np.std(samples, axis = 1)

    # evaluate genmodel 
    print('evaluate genmodel')
    for alpha in alpha_grid:
        # evaluate density model's conditional/marginal coverage
        Y_lo = np.quantile(samples, alpha/2, axis = 1)
        Y_up = np.quantile(samples, 1 - alpha/2, axis = 1)

        marginal_coverage = np.mean((Y_lo <= Ytest) & (Ytest <= Y_up))
        print(f'marginal coverage of density model is {marginal_coverage}')

        cover = (Y_lo <= Ytest) & (Ytest <= Y_up)
        cc = 1
        for eachs in set(sen):
            cc = min(cc, cover[sen == eachs].mean())

        conditional_coverage.append(cc)
        print(f'Average conditional coverage of density model is {cc}')
        coverage.append(np.mean((Y_lo <= Ytest) & (Ytest <= Y_up)))
        set_size.append(np.mean(Y_up - Y_lo))

        pred = np.concatenate([Y_lo.reshape(-1,1), Y_up.reshape(-1,1)], axis = 1)
        wslab = wsc_unbiased(Xtest, Ytest, pred, delta=args.wsc_delta, M=max(200,Xtest.shape[1]+50), test_size=0.5, random_state=2020, verbose=False) 
        print(f'wsc of density model is {wslab}')
        est_mean = np.mean(samples, axis = 1)

        mdn_results = pd.concat([mdn_results, pd.DataFrame({"iter": [iter], "alpha": [alpha], "coverage": [np.mean((Y_lo <= Ytest) & (Ytest <= Y_up))], \
        "set_size": [np.mean(Y_up - Y_lo)], "conditional_coverage": [np.min(cc)], 'mse': [((Ytest - est_mean)**2).mean()], 
        'wslab': [wslab]})])
    print('evaluate genmodel done.')

    mdn_results.to_csv(f'./log/{args.dataset}_{args.densitymodel}_genmodel_coverage.csv', index=False)
    mdn_results_mean = mdn_results.groupby(['alpha']).mean().reset_index()
    mdn_results_se = mdn_results.groupby(['alpha']).std().reset_index() / np.sqrt(5)
    mdn_results_se.columns = [each + '_se' for each in mdn_results_se.columns]
    mdn_results_sum = pd.concat([mdn_results_mean, mdn_results_se], axis = 1)
    mdn_results_sum.to_csv(f'./log/{args.dataset}_{args.densitymodel}_genmodel_summary.csv', index=False)

    # evaluate basemodel 
    Ymodel_base = basemodel.fit(X, Y)
    Yhat = Ymodel_base.predict(Xval)
    Yscore = conformalScore(Yval, Yhat, sd = val_std)
    Yhat_test = Ymodel_base.predict(Xtest)
    for alpha in alpha_grid:
        qhat = np.quantile(Yscore, np.ceil((nval+1)*(1-alpha))/nval, interpolation='higher')
        Yslack = qhat

        print(f'cutoff is {Yslack}')
        if args.conformalscore == 'residual':
            Ylo = Yhat_test - Yslack
            Yup = Yhat_test + Yslack
        elif args.conformalscore == 'normalized':
            Ylo = Yhat_test - Yslack * test_std
            Yup = Yhat_test + Yslack * test_std

        CI = pd.DataFrame({"lower": Ylo.reshape(-1), "upper": Yup.reshape(-1)})

        # check subgroup coverage
        cover = (Ylo <= Ytest) & (Ytest <= Yup)
        cc = 1
        for eachs in set(sen):
            cc = min(cc, cover[sen == eachs].mean())

        conditional_coverage.append(cc)
        print(f'Coverage is {np.mean((Ylo <= Ytest) & (Ytest <= Yup))}')
        print(f'Average coverage set size is {np.mean(Yup - Ylo)}')
        print(f'Average conditional coverage is {cc}')
        coverage.append(np.mean((Ylo <= Ytest) & (Ytest <= Yup)))
        set_size.append(np.mean(Yup - Ylo))

        pred = np.concatenate([Ylo.reshape(-1,1), Yup.reshape(-1,1)], axis = 1)
        wslab = wsc_unbiased(Xtest, Ytest, pred, delta=args.wsc_delta, M=max(200,Xtest.shape[1]+50), test_size=0.5, random_state=2020, verbose=False)
        print(f'wsc is {wslab}')

        covered_index = ((Ylo <= Ytest) & (Ytest <= Yup))
        base_results = pd.concat([base_results, pd.DataFrame({"iter": [iter], "alpha": [alpha], "coverage": [np.mean((Ylo <= Ytest) & (Ytest <= Yup))], \
        "set_size": [np.mean(Yup - Ylo)], "conditional_coverage": [np.min(cc)], 'mse': [((Ytest - Yhat_test)**2).mean()], 
        "wslab": [wslab]})])

        filename = f'./log/{args.dataset}_{args.model}_{args.densitymodel}_{args.conformalscore}_base_coverage.csv'
        filename = filename.replace('_cc', '')
        base_results.to_csv(filename, index=False)
        base_results_mean = base_results.groupby(['alpha']).mean().reset_index()
        base_results_se = base_results.groupby(['alpha']).std().reset_index() / np.sqrt(5)
        base_results_se.columns = [each + '_se' for each in base_results_se.columns]
        base_results_sum = pd.concat([base_results_mean, base_results_se], axis = 1)
        filename = f'./log/{args.dataset}_{args.model}_{args.densitymodel}_{args.conformalscore}_base_summary.csv'
        filename = filename.replace('_cc', '')
        base_results_sum.to_csv(filename, index=False)

if model == 'linear_cc':
    linearmodel_cc = TorchLinearModel_CC(X.shape[1], lamb=args.lamb, density_model = density_model)
    Ymodel = linearmodel_cc.fit(X, Y, Xval, Yval, Xtest, Ytest, sen, i = iter)
elif model == 'mlp_cc':
    linearmodel_cc = TorchLinearModel_CC(X.shape[1], lamb=args.lamb, density_model = density_model)
    Ymodel = linearmodel_cc.fit(X, Y, Xval, Yval, Xtest, Ytest, sen, i = iter)
elif model == 'linear' or model == 'mlp':
    linearmodel = TorchBaseModel(X.shape[1])
    Ymodel = linearmodel.fit(X, Y)

Yhat = Ymodel.predict(Xval)
Yscore = conformalScore(Yval, Yhat, sd = val_std)
Yhat_test = Ymodel.predict(Xtest)

alpha = 0.1 
qhat = np.quantile(Yscore, np.ceil((nval+1)*(1-alpha))/nval, interpolation='higher')
Yslack = qhat

print(f'cutoff is {Yslack}')
if args.conformalscore == 'residual':
    Ylo = Yhat_test - Yslack
    Yup = Yhat_test + Yslack
elif args.conformalscore == 'normalized':
    Ylo = Yhat_test - Yslack * test_std
    Yup = Yhat_test + Yslack * test_std

CI = pd.DataFrame({"lower": Ylo.reshape(-1), "upper": Yup.reshape(-1)})

# check subgroup coverage
cover = (Ylo <= Ytest) & (Ytest <= Yup)
cc = 1
for eachs in set(sen):
    cc = min(cc, cover[sen == eachs].mean())

conditional_coverage.append(cc)
print(f'Coverage is {np.mean((Ylo <= Ytest) & (Ytest <= Yup))}')
print(f'Average coverage set size is {np.mean(Yup - Ylo)}')
print(f'Average conditional coverage is {cc}')
coverage.append(np.mean((Ylo <= Ytest) & (Ytest <= Yup)))
set_size.append(np.mean(Yup - Ylo))

pred = np.concatenate([Ylo.reshape(-1,1), Yup.reshape(-1,1)], axis = 1)
wslab = wsc_unbiased(Xtest, Ytest, pred, delta=0.1, M=max(200,Xtest.shape[1]+50), test_size=0.5, random_state=2020, verbose=False)
print(f'wsc is {wslab}')

covered_index = ((Ylo <= Ytest) & (Ytest <= Yup))
results = pd.concat([results, pd.DataFrame({"iter": [iter], "alpha": [alpha], "coverage": [np.mean((Ylo <= Ytest) & (Ytest <= Yup))], \
"set_size": [np.mean(Yup - Ylo)], "conditional_coverage": [np.min(cc)], 'mse': [((Ytest - Yhat_test)**2).mean()], 
"wslab": [wslab]})])

results.to_csv(f'./log/{args.dataset}_{args.model}_{args.lamb}_{args.densitymodel}_{args.conformalscore}_coverage.csv', index=False)

result_mean = results.groupby(['alpha']).mean().reset_index()
result_se = results.groupby(['alpha']).std().reset_index() / np.sqrt(5)
result_se.columns = [each + '_se' for each in result_se.columns]
result_sum = pd.concat([result_mean, result_se], axis = 1)
result_sum.to_csv(f'./log/{args.dataset}_{args.model}_{args.lamb}_{args.densitymodel}_{args.conformalscore}_coverage_summary.csv', index=False)
    
print(f'WSLAB is {wslab}')

dataset size is 10428, number of features is 139.
train size is 6256, val size is 2085, test size is 2087.
density model fitting done.
99 0.8078312873840332
199 0.6219979524612427
299 0.590796172618866
399 0.5858684182167053
499 0.5849030017852783
599 0.5845747590065002
699 0.5844173431396484
799 0.5843259692192078
899 0.5842612385749817
999 0.5842065215110779
1099 0.5841558575630188
1199 0.5841073393821716
1299 0.5840607285499573
1399 0.5840161442756653
1499 0.5839739441871643
1599 0.5839343667030334
1699 0.5838974714279175
1799 0.5838636159896851
1899 0.5838327407836914
1999 0.5838049650192261
2099 0.5837801098823547
2199 0.5837582349777222
2299 0.5837391018867493
2399 0.5837225317955017
2499 0.5837084650993347
2599 0.5836965441703796
2699 0.5836866497993469
2799 0.5836784839630127
2899 0.5836718678474426
2999 0.5836665630340576
3099 0.5836623311042786
3199 0.5836589932441711
3299 0.5836564898490906
3399 0.583654522895813
3499 0.5836530923843384
3599 0.5836520195007324
3699 0.5836512

/tmp/ipykernel_104074/2293351234.py:177: DeprecationWarning: the `interpolation=` argument to quantile was renamed to `method=`, which has additional options.
Users of the modes 'nearest', 'lower', 'higher', or 'midpoint' are encouraged to review the method they used. (Deprecated NumPy 1.22)
  qhat = np.quantile(Yscore, np.ceil((nval+1)*(1-alpha))/nval, interpolation='higher')


wsc is 0.7313432835820896
WSLAB is 0.7313432835820896


/tmp/ipykernel_104074/2293351234.py:214: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  result_mean = results.groupby(['alpha']).mean().reset_index()
/tmp/ipykernel_104074/2293351234.py:215: FutureWarning: The default value of numeric_only in DataFrameGroupBy.std is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  result_se = results.groupby(['alpha']).std().reset_index() / np.sqrt(5)


In [4]:
print(f'WSLAB is {wslab}')

WSLAB is 0.7313432835820896


In [5]:
# mlp cc model 
seed_everything(0)
X, Y = GetDataset(dataset_name, './data/')
Y += 1e-3 * np.random.normal(size=Y.shape)

args.model = 'mlp_cc'
model = 'mlp_cc'
basemodel = TorchBaseModel(X.shape[1])

index = np.random.permutation(X.shape[0])
print(f'dataset size is {X.shape[0]}, number of features is {X.shape[1]}.')
nval = int(X.shape[0] * 0.2)
ntrain = int(X.shape[0] * 0.6)
Xval = X[index[:nval],:]
Yval = Y[index[:nval]]
Xtest = X[index[(nval+ntrain):],:]
Ytest = Y[index[(nval+ntrain):]]
X = X[index[(nval):(nval+ntrain)],:]
Y = Y[index[(nval):(nval+ntrain)]]
print(f'train size is {X.shape[0]}, val size is {Xval.shape[0]}, test size is {Xtest.shape[0]}.')
scaler = StandardScaler()
scaler.fit(X)

y_scaler = StandardScaler()
y_scaler.fit(Y.reshape(-1,1))

Y = y_scaler.transform(Y.reshape(-1,1)).reshape(-1)
Yval = y_scaler.transform(Yval.reshape(-1,1)).reshape(-1)
Ytest = y_scaler.transform(Ytest.reshape(-1,1)).reshape(-1)

if dataset_name == 'meps_19' or dataset_name == 'meps_20' or dataset_name == 'meps_21':
    # define a subgroup - checking some surrogate variable
    sen = np.zeros_like(Ytest)
    sen[(Xtest[:, 9] == 1) & (Xtest[:, -1] == 1)] = 0
    sen[(Xtest[:, 9] == 1) & (Xtest[:, -1] == 0)] = 1
    sen[(Xtest[:, 9] == 0) & (Xtest[:, -1] == 1)] = 2
    sen[(Xtest[:, 9] == 0) & (Xtest[:, -1] == 0)] = 3
    
X = scaler.transform(X)
Xval = scaler.transform(Xval)
Xtest = scaler.transform(Xtest)

if (model == 'linear' or model == 'mlp') and args.conformalscore == 'residual':
    pass 
elif args.densitymodel == 'CVAE':
    density_model = CVAE(n_epochs=500, latent_dim=8, hidden = (32,32), batch_size=128, lr=1e-3, weight_decay=1e-4, verbose = 1)
    density_model.fit(Y.reshape(-1,1), X)

print('density model fitting done.')
nval = Xval.shape[0]

nsamples = 1000

if (model == 'linear' or model == 'mlp') and args.conformalscore == 'residual':
    val_std = 1
    test_std = 1
else:
    samples_val = np.zeros((nval, nsamples))
    for i in range(nsamples):
        #samples_val[:,i] = density_model.sample(Xval)[1].reshape(-1)
        #samples_val[:,i] = density_model.generate(Xval.shape[0], cond = Xval).values.reshape(-1)
        samples_val[:,i] = density_model.sample(Xval).reshape(-1)

    val_std = np.std(samples_val, axis = 1)

    ntest = Xtest.shape[0]
    samples = np.zeros((ntest, nsamples))
    for i in range(nsamples):
        #samples[:,i] = density_model.sample(Xtest)[1].reshape(-1)
        #samples[:,i] = density_model.generate(Xtest.shape[0], cond = Xtest).values.reshape(-1)
        samples[:,i] = density_model.sample(Xtest).reshape(-1)

    test_std = np.std(samples, axis = 1)

    # evaluate genmodel 
    print('evaluate genmodel')
    for alpha in alpha_grid:
        # evaluate density model's conditional/marginal coverage
        Y_lo = np.quantile(samples, alpha/2, axis = 1)
        Y_up = np.quantile(samples, 1 - alpha/2, axis = 1)

        marginal_coverage = np.mean((Y_lo <= Ytest) & (Ytest <= Y_up))
        print(f'marginal coverage of density model is {marginal_coverage}')

        cover = (Y_lo <= Ytest) & (Ytest <= Y_up)
        cc = 1
        for eachs in set(sen):
            cc = min(cc, cover[sen == eachs].mean())

        conditional_coverage.append(cc)
        print(f'Average conditional coverage of density model is {cc}')
        coverage.append(np.mean((Y_lo <= Ytest) & (Ytest <= Y_up)))
        set_size.append(np.mean(Y_up - Y_lo))

        pred = np.concatenate([Y_lo.reshape(-1,1), Y_up.reshape(-1,1)], axis = 1)
        wslab = wsc_unbiased(Xtest, Ytest, pred, delta=args.wsc_delta, M=max(200,Xtest.shape[1]+50), test_size=0.5, random_state=2020, verbose=False) 
        print(f'wsc of density model is {wslab}')
        est_mean = np.mean(samples, axis = 1)

        mdn_results = pd.concat([mdn_results, pd.DataFrame({"iter": [iter], "alpha": [alpha], "coverage": [np.mean((Y_lo <= Ytest) & (Ytest <= Y_up))], \
        "set_size": [np.mean(Y_up - Y_lo)], "conditional_coverage": [np.min(cc)], 'mse': [((Ytest - est_mean)**2).mean()], 
        'wslab': [wslab]})])
    print('evaluate genmodel done.')

    mdn_results.to_csv(f'./log/{args.dataset}_{args.densitymodel}_genmodel_coverage.csv', index=False)
    mdn_results_mean = mdn_results.groupby(['alpha']).mean().reset_index()
    mdn_results_se = mdn_results.groupby(['alpha']).std().reset_index() / np.sqrt(5)
    mdn_results_se.columns = [each + '_se' for each in mdn_results_se.columns]
    mdn_results_sum = pd.concat([mdn_results_mean, mdn_results_se], axis = 1)
    mdn_results_sum.to_csv(f'./log/{args.dataset}_{args.densitymodel}_genmodel_summary.csv', index=False)

    # evaluate basemodel 
    Ymodel_base = basemodel.fit(X, Y)
    Yhat = Ymodel_base.predict(Xval)
    Yscore = conformalScore(Yval, Yhat, sd = val_std)
    Yhat_test = Ymodel_base.predict(Xtest)
    for alpha in alpha_grid:
        qhat = np.quantile(Yscore, np.ceil((nval+1)*(1-alpha))/nval, interpolation='higher')
        Yslack = qhat

        print(f'cutoff is {Yslack}')
        if args.conformalscore == 'residual':
            Ylo = Yhat_test - Yslack
            Yup = Yhat_test + Yslack
        elif args.conformalscore == 'normalized':
            Ylo = Yhat_test - Yslack * test_std
            Yup = Yhat_test + Yslack * test_std

        CI = pd.DataFrame({"lower": Ylo.reshape(-1), "upper": Yup.reshape(-1)})

        # check subgroup coverage
        cover = (Ylo <= Ytest) & (Ytest <= Yup)
        cc = 1
        for eachs in set(sen):
            cc = min(cc, cover[sen == eachs].mean())

        conditional_coverage.append(cc)
        print(f'Coverage is {np.mean((Ylo <= Ytest) & (Ytest <= Yup))}')
        print(f'Average coverage set size is {np.mean(Yup - Ylo)}')
        print(f'Average conditional coverage is {cc}')
        coverage.append(np.mean((Ylo <= Ytest) & (Ytest <= Yup)))
        set_size.append(np.mean(Yup - Ylo))

        pred = np.concatenate([Ylo.reshape(-1,1), Yup.reshape(-1,1)], axis = 1)
        wslab = wsc_unbiased(Xtest, Ytest, pred, delta=args.wsc_delta, M=max(200,Xtest.shape[1]+50), test_size=0.5, random_state=2020, verbose=False)
        print(f'wsc is {wslab}')

        covered_index = ((Ylo <= Ytest) & (Ytest <= Yup))
        base_results = pd.concat([base_results, pd.DataFrame({"iter": [iter], "alpha": [alpha], "coverage": [np.mean((Ylo <= Ytest) & (Ytest <= Yup))], \
        "set_size": [np.mean(Yup - Ylo)], "conditional_coverage": [np.min(cc)], 'mse': [((Ytest - Yhat_test)**2).mean()], 
        "wslab": [wslab]})])

        filename = f'./log/{args.dataset}_{args.model}_{args.densitymodel}_{args.conformalscore}_base_coverage.csv'
        filename = filename.replace('_cc', '')
        base_results.to_csv(filename, index=False)
        base_results_mean = base_results.groupby(['alpha']).mean().reset_index()
        base_results_se = base_results.groupby(['alpha']).std().reset_index() / np.sqrt(5)
        base_results_se.columns = [each + '_se' for each in base_results_se.columns]
        base_results_sum = pd.concat([base_results_mean, base_results_se], axis = 1)
        filename = f'./log/{args.dataset}_{args.model}_{args.densitymodel}_{args.conformalscore}_base_summary.csv'
        filename = filename.replace('_cc', '')
        base_results_sum.to_csv(filename, index=False)

if model == 'linear_cc':
    linearmodel_cc = TorchLinearModel_CC(X.shape[1], lamb=args.lamb, density_model = density_model)
    Ymodel = linearmodel_cc.fit(X, Y, Xval, Yval, Xtest, Ytest, sen, i = iter)
elif model == 'mlp_cc':
    linearmodel_cc = TorchLinearModel_CC(X.shape[1], lamb=args.lamb, density_model = density_model)
    Ymodel = linearmodel_cc.fit(X, Y, Xval, Yval, Xtest, Ytest, sen, i = iter)
elif model == 'linear' or model == 'mlp':
    linearmodel = TorchBaseModel(X.shape[1])
    Ymodel = linearmodel.fit(X, Y)

Yhat = Ymodel.predict(Xval)
Yscore = conformalScore(Yval, Yhat, sd = val_std)
Yhat_test = Ymodel.predict(Xtest)

alpha = 0.1 
qhat = np.quantile(Yscore, np.ceil((nval+1)*(1-alpha))/nval, interpolation='higher')
Yslack = qhat

print(f'cutoff is {Yslack}')
if args.conformalscore == 'residual':
    Ylo = Yhat_test - Yslack
    Yup = Yhat_test + Yslack
elif args.conformalscore == 'normalized':
    Ylo = Yhat_test - Yslack * test_std
    Yup = Yhat_test + Yslack * test_std

CI = pd.DataFrame({"lower": Ylo.reshape(-1), "upper": Yup.reshape(-1)})

# check subgroup coverage
cover = (Ylo <= Ytest) & (Ytest <= Yup)
cc = 1
for eachs in set(sen):
    cc = min(cc, cover[sen == eachs].mean())

conditional_coverage.append(cc)
print(f'Coverage is {np.mean((Ylo <= Ytest) & (Ytest <= Yup))}')
print(f'Average coverage set size is {np.mean(Yup - Ylo)}')
print(f'Average conditional coverage is {cc}')
coverage.append(np.mean((Ylo <= Ytest) & (Ytest <= Yup)))
set_size.append(np.mean(Yup - Ylo))

pred = np.concatenate([Ylo.reshape(-1,1), Yup.reshape(-1,1)], axis = 1)
wslab = wsc_unbiased(Xtest, Ytest, pred, delta=0.1, M=max(200,Xtest.shape[1]+50), test_size=0.5, random_state=2020, verbose=False)
print(f'wsc is {wslab}')

covered_index = ((Ylo <= Ytest) & (Ytest <= Yup))
results = pd.concat([results, pd.DataFrame({"iter": [iter], "alpha": [alpha], "coverage": [np.mean((Ylo <= Ytest) & (Ytest <= Yup))], \
"set_size": [np.mean(Yup - Ylo)], "conditional_coverage": [np.min(cc)], 'mse': [((Ytest - Yhat_test)**2).mean()], 
"wslab": [wslab]})])

results.to_csv(f'./log/{args.dataset}_{args.model}_{args.lamb}_{args.densitymodel}_{args.conformalscore}_coverage.csv', index=False)

result_mean = results.groupby(['alpha']).mean().reset_index()
result_se = results.groupby(['alpha']).std().reset_index() / np.sqrt(5)
result_se.columns = [each + '_se' for each in result_se.columns]
result_sum = pd.concat([result_mean, result_se], axis = 1)
result_sum.to_csv(f'./log/{args.dataset}_{args.model}_{args.lamb}_{args.densitymodel}_{args.conformalscore}_coverage_summary.csv', index=False)
    
print(f'WSLAB is {wslab}')

dataset size is 10428, number of features is 139.
train size is 6256, val size is 2085, test size is 2087.
epoch: 0, loss: 35.2357
epoch: 10, loss: 1.3820
epoch: 20, loss: 0.7988
epoch: 30, loss: 0.5346
epoch: 40, loss: 0.5216
epoch: 50, loss: 0.4538
epoch: 60, loss: 0.4164
epoch: 70, loss: 0.4289
epoch: 80, loss: 0.4257
epoch: 90, loss: 0.4030
epoch: 100, loss: 0.3966
epoch: 110, loss: 0.3960
epoch: 120, loss: 0.3836
epoch: 130, loss: 0.3299
epoch: 140, loss: 0.2795
epoch: 150, loss: 0.2785
epoch: 160, loss: 0.2731
epoch: 170, loss: 0.2753
epoch: 180, loss: 0.2751
epoch: 190, loss: 0.2585
epoch: 200, loss: 0.2606
epoch: 210, loss: 0.2994
epoch: 220, loss: 0.3157
epoch: 230, loss: 0.2671
epoch: 240, loss: 0.2478
epoch: 250, loss: 0.2553
epoch: 260, loss: 0.2983
epoch: 270, loss: 0.2686
epoch: 280, loss: 0.2472
epoch: 290, loss: 0.2703
epoch: 300, loss: 0.2586
epoch: 310, loss: 0.2630
epoch: 320, loss: 0.2637
epoch: 330, loss: 0.2454
epoch: 340, loss: 0.2578
epoch: 350, loss: 0.2458
epo

/tmp/ipykernel_104074/331148753.py:106: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  mdn_results_mean = mdn_results.groupby(['alpha']).mean().reset_index()
/tmp/ipykernel_104074/331148753.py:107: FutureWarning: The default value of numeric_only in DataFrameGroupBy.std is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  mdn_results_se = mdn_results.groupby(['alpha']).std().reset_index() / np.sqrt(5)


99 0.563853919506073
199 0.50897616147995
299 0.47220614552497864
399 0.44106656312942505
499 0.41305670142173767
599 0.3930303156375885
699 0.3775741159915924
799 0.36474114656448364
899 0.35359087586402893
999 0.3456641435623169
1099 0.33952006697654724
1199 0.33527156710624695
1299 0.3314869999885559
1399 0.32842782139778137
1499 0.3262264132499695
1599 0.3243827819824219
1699 0.3223129212856293
1799 0.3209473490715027
1899 0.31955626606941223
1999 0.31765642762184143
2099 0.31645530462265015
2199 0.31551143527030945
2299 0.314611554145813
2399 0.31338655948638916
2499 0.31208786368370056
2599 0.31100520491600037
2699 0.30977195501327515
2799 0.3085612654685974
2899 0.30770212411880493
2999 0.3068595826625824
3099 0.30605289340019226
3199 0.3051958680152893
3299 0.3046673834323883
3399 0.30421045422554016
3499 0.30373501777648926
3599 0.3032630383968353
3699 0.3029496371746063
3799 0.30262526869773865
3899 0.3022840917110443
3999 0.301984041929245
4099 0.3014705181121826
4199 0.3012

/tmp/ipykernel_104074/331148753.py:118: DeprecationWarning: the `interpolation=` argument to quantile was renamed to `method=`, which has additional options.
Users of the modes 'nearest', 'lower', 'higher', or 'midpoint' are encouraged to review the method they used. (Deprecated NumPy 1.22)
  qhat = np.quantile(Yscore, np.ceil((nval+1)*(1-alpha))/nval, interpolation='higher')


wsc is 0.7319587628865979
cutoff is 1.2247447967529297
Coverage is 0.8217537134643028
Average coverage set size is 2.4494895935058594
Average conditional coverage is 0.6296296296296297


...

/tmp/ipykernel_104074/1767092640.py:253: DeprecationWarning: the `interpolation=` argument to quantile was renamed to `method=`, which has additional options.
Users of the modes 'nearest', 'lower', 'higher', or 'midpoint' are encouraged to review the method they used. (Deprecated NumPy 1.22)
  qhat = np.quantile(Yscore, np.ceil((nval+1)*(1-alpha))/nval, interpolation='higher')


cutoff is 2.812591552734375
59 3.344215154647827
mc is 0.8941063727839004
cc is 0.8820276497695853
wsc is 0.8561643835616438
cutoff is 2.3128604888916016
Coverage is 0.900335409678965
Average coverage set size is 4.625720977783203
Average conditional coverage is 0.8518518518518519


/tmp/ipykernel_104074/331148753.py:179: DeprecationWarning: the `interpolation=` argument to quantile was renamed to `method=`, which has additional options.
Users of the modes 'nearest', 'lower', 'higher', or 'midpoint' are encouraged to review the method they used. (Deprecated NumPy 1.22)
  qhat = np.quantile(Yscore, np.ceil((nval+1)*(1-alpha))/nval, interpolation='higher')


wsc is 0.9075144508670521
WSLAB is 0.9075144508670521


/tmp/ipykernel_104074/331148753.py:216: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  result_mean = results.groupby(['alpha']).mean().reset_index()
/tmp/ipykernel_104074/331148753.py:217: FutureWarning: The default value of numeric_only in DataFrameGroupBy.std is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  result_se = results.groupby(['alpha']).std().reset_index() / np.sqrt(5)


In [6]:
print(f'WSLAB is {wslab}')

WSLAB is 0.9075144508670521
